In [40]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from openai import OpenAI
from key import OPENAI_API_KEY
client = OpenAI(api_key=OPENAI_API_KEY)

In [41]:
# read the responses from the json file
def read_responses(output_file):
    responses_recommendations = []
    with open(output_file, 'r') as f:
        for line in f:
            responses_recommendations.append(eval(line))
    return responses_recommendations

# calculate the hit rate
# hit rate for the recommendations
def hit_rate(recommendation, actual):
    hit = 0
    for rec in recommendation:
        if rec in actual:
            hit += 1
    return hit/len(recommendation)

# NDCG for the recommendations
def ndcg(recommendation, actual):
    dcg = 0
    for i, rec in enumerate(recommendation):
        if rec in actual:
            dcg += 1/np.log2(i+2)
    idcg = 0
    for i in range(len(actual)):
        idcg += 1/np.log2(i+2)
    return dcg/idcg


def eval_response(responses_recommendations, selected_movie_data, original_movie_data, num_users):
    hit_rates = []
    ndcg_scores = []
    for i in range(num_users):
        user_id = selected_movie_data['userId'].unique()[i]
        user_movies = original_movie_data[original_movie_data['userId']==user_id]
        user_watched_movies = user_movies['title'].values
        # print(f"User {user_id} watched movies: {user_watched_movies}")
        recommendation = responses_recommendations[i]
        # print(f"Recommendation: {recommendation}")
        hit_rate_ = hit_rate(recommendation, user_watched_movies)
        # print(f"Hit Rate: {hit_rate_}")
        hit_rates.append(hit_rate_)
        ndcg_ = ndcg(recommendation, user_watched_movies)
        # print(f"NDCG: {ndcg_}")
        ndcg_scores.append(ndcg_)

    avg_hit_rate = np.mean(hit_rates)
    avg_ndcg = np.mean(ndcg_scores)

    print(f"Average Hit Rate: {avg_hit_rate}")
    print(f"Average NDCG: {avg_ndcg}")

    return hit_rates, ndcg_scores

In [42]:
movie_latest_small=pd.read_csv('../../data/processed_movie_latest_small.csv')

# for each user, find his top-most rated movies
N_TOP_MOVIES = 10
def get_top_recent_movies(movies_df, user_id, num_movies=N_TOP_MOVIES):
    user_ratings = movies_df[movies_df['userId'] == user_id]
    user_ratings = user_ratings.sort_values(by=['rating', 'timestamp'], ascending=[False, False])
    return user_ratings.head(num_movies)

# get the top 10 movies for all users
top_movies = pd.DataFrame()
for user_id in movie_latest_small['userId'].unique():
    top_movies = pd.concat([top_movies, get_top_recent_movies(movie_latest_small, user_id, 10)])
top_movies.reset_index(drop=True, inplace=True)

# exclude the top 5 movies from top_movies from movie_latest_small
for user in movie_latest_small['userId'].unique():
    top_movies_user = top_movies[top_movies['userId'] == user]
    top_movies_user_titles = top_movies_user['title'].values
    movie_exclude = movie_latest_small[(movie_latest_small['userId'] == user) & (movie_latest_small['title'].isin(top_movies_user_titles))]
    movie_latest_small_2 = movie_latest_small.drop(movie_exclude.index)
movie_latest_small_2.reset_index(drop=True, inplace=True)

In [43]:
# num_users = 5

# Uncomment below to run on all users
num_users = len(movie_latest_small['userId'].unique())

print(f"Running the recommender on {num_users} users")

Running the recommender on 610 users


In [44]:
avg_hit_rate_basic_list = []
avg_hit_rate_genre_list = []
avg_hit_rate_summary_list = []
avg_hit_rate_similar_user_10_20_list = []
avg_hit_rate_role_list = []
avg_hit_rate_2step_list = []

avg_ndcg_genre_list = []
avg_ndcg_basic_list = []
avg_ndcg_summary_list = []
avg_ndcg_similar_user_10_20_list = []
avg_ndcg_role_list = []
avg_ndcg_2step_list = []

for i in range(1,4):
    output_basic_file = f"responses_basic_{i}.json"
    output_genre_file = f"responses_genre_{i}.json"
    output_summary_file = f"responses_add_summary_{i}.json"
    output_similar_user_10_20_file = f"responses_similar_user_10_20_{i}.json"
    output_role_file = f"responses_add_role_{i}.json"
    output_2step_file = f"responses_2step_{i}.json"

    responses_recommendations_basic = read_responses(output_basic_file)
    responses_recommendations_genre = read_responses(output_genre_file)
    responses_recommendations_summary = read_responses(output_summary_file)
    responses_recommendations_similar_user_10_20 = read_responses(output_similar_user_10_20_file)
    responses_recommendations_role = read_responses(output_role_file)
    responses_recommendations_2step = read_responses(output_2step_file)

    hit_rate_basic, ndcg_basic = eval_response(responses_recommendations_basic, top_movies, movie_latest_small, num_users)
    avg_hit_rate_basic_list.append(np.mean(hit_rate_basic))
    avg_ndcg_basic_list.append(np.mean(ndcg_basic))

    hit_rate_genre, ndcg_genre = eval_response(responses_recommendations_genre, top_movies, movie_latest_small, num_users)
    avg_hit_rate_genre_list.append(np.mean(hit_rate_genre))
    avg_ndcg_genre_list.append(np.mean(ndcg_genre))

    hit_rate_summary, ndcg_summary = eval_response(responses_recommendations_summary, top_movies, movie_latest_small, num_users)
    avg_hit_rate_summary_list.append(np.mean(hit_rate_summary))
    avg_ndcg_summary_list.append(np.mean(ndcg_summary))

    hit_rate_similar_user_10_20, ndcg_similar_user_10_20 = eval_response(responses_recommendations_similar_user_10_20, top_movies, movie_latest_small, num_users)
    avg_hit_rate_similar_user_10_20_list.append(np.mean(hit_rate_similar_user_10_20))
    avg_ndcg_similar_user_10_20_list.append(np.mean(ndcg_similar_user_10_20))

    hit_rate_role, ndcg_role = eval_response(responses_recommendations_role, top_movies, movie_latest_small, num_users)
    avg_hit_rate_role_list.append(np.mean(hit_rate_role))
    avg_ndcg_role_list.append(np.mean(ndcg_role))

    hit_rate_2step, ndcg_2step = eval_response(responses_recommendations_2step, top_movies, movie_latest_small, num_users)
    avg_hit_rate_2step_list.append(np.mean(hit_rate_2step))
    avg_ndcg_2step_list.append(np.mean(ndcg_2step))

# make a dataframe of the hit rates and ndcg scores
hit_rate_df = pd.DataFrame()
hit_rate_df['avg_hit_rate_basic'] = avg_hit_rate_basic_list
hit_rate_df['avg_hit_rate_genre'] = avg_hit_rate_genre_list
hit_rate_df['avg_hit_rate_summary'] = avg_hit_rate_summary_list
hit_rate_df['avg_hit_rate_similar_user_10_20'] = avg_hit_rate_similar_user_10_20_list
hit_rate_df['avg_hit_rate_role'] = avg_hit_rate_role_list
hit_rate_df['avg_hit_rate_2step'] = avg_hit_rate_2step_list

ndcg_df = pd.DataFrame()
ndcg_df['avg_ndcg_basic'] = avg_ndcg_basic_list
ndcg_df['avg_ndcg_genre'] = avg_ndcg_genre_list
ndcg_df['avg_ndcg_summary'] = avg_ndcg_summary_list
ndcg_df['avg_ndcg_similar_user_10_20'] = avg_ndcg_similar_user_10_20_list
ndcg_df['avg_ndcg_role'] = avg_ndcg_role_list
ndcg_df['avg_ndcg_2step'] = avg_ndcg_2step_list

Average Hit Rate: 0.13081967213114754
Average NDCG: 0.03116397800190152
Average Hit Rate: 0.13081967213114756
Average NDCG: 0.029897241177440213
Average Hit Rate: 0.13688524590163934
Average NDCG: 0.02961337200616843
Average Hit Rate: 0.14278688524590163
Average NDCG: 0.03217651576059022
Average Hit Rate: 0.17508196721311478
Average NDCG: 0.04420126784731338
Average Hit Rate: 0.13262295081967215
Average NDCG: 0.03127734261573137
Average Hit Rate: 0.12925318761384336
Average NDCG: 0.029866383697874685
Average Hit Rate: 0.13983606557377048
Average NDCG: 0.029895005871044593
Average Hit Rate: 0.14196721311475408
Average NDCG: 0.03237182627450347
Average Hit Rate: 0.17459016393442625
Average NDCG: 0.04400102423663841
Average Hit Rate: 0.14885245901639343
Average NDCG: 0.036809731788525875
Average Hit Rate: 0.1301639344262295
Average NDCG: 0.03035309645173963
Average Hit Rate: 0.1378688524590164
Average NDCG: 0.029681742349869932
Average Hit Rate: 0.14311475409836066
Average NDCG: 0.0333771

In [45]:
hit_rate_df.describe()

,avg_hit_rate_basic,avg_hit_rate_genre,avg_hit_rate_summary,avg_hit_rate_similar_user_10_20,avg_hit_rate_role
count,3.000000,3.000000,3.000000,3.000000,3.000000
mean,0.137432,0.130079,0.138197,0.142623,0.175137
std,0.009932,0.000787,0.001502,0.000591,0.000576
min,0.130820,0.129253,0.136885,0.141967,0.174590
25%,0.131721,0.129709,0.137377,0.142377,0.174836
50%,0.132623,0.130164,0.137869,0.142787,0.175082
75%,0.140738,0.130492,0.138852,0.142951,0.175410
max,0.148852,0.130820,0.139836,0.143115,0.175738


In [46]:
ndcg_df.describe()

,avg_ndcg_basic,avg_ndcg_genre,avg_ndcg_summary,avg_ndcg_similar_user_10_20,avg_ndcg_role
count,3.000000,3.000000,3.000000,3.000000,3.000000
mean,0.033084,0.030039,0.029730,0.032642,0.044198
std,0.003227,0.000273,0.000147,0.000644,0.000195
min,0.031164,0.029866,0.029613,0.032177,0.044001
25%,0.031221,0.029882,0.029648,0.032274,0.044101
50%,0.031277,0.029897,0.029682,0.032372,0.044201
75%,0.034044,0.030125,0.029788,0.032875,0.044296
max,0.036810,0.030353,0.029895,0.033377,0.044391
